In [164]:
#Dependencies
import tweepy
import json
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

In [165]:
#Import and Initialize Sentiment Analyzer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [166]:
#Twitter API Keys
consumer_key = "vOppZRR2bYoErztWlHIYfGzle"
consumer_secret = "IBErpFzMsZCTUkGohAt7TLbC1rwYYo9ltvcovbM2nUHgcxayOc"
access_token = "937018806580297728-CCaeIjsqPR05j6XC8cs3xMtUght2TZt"
access_token_secret = "6ttwxnr5RgdPJpgwm3iFxp0fMtnl6PWkU55x1ABcmvj7m"

In [167]:
#Setup Tweepy API Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

In [168]:
#Create list that holds news outlet twitter user account names
news_outlets = ["@BBC","@CBS", "@CNN", "@FoxNews", "@nytimes"]

In [169]:
#Create API call to search for the last 100 tweets posted by each media outlet to review parameters
public_tweets = api.user_timeline(news_outlets, count=1)
public_tweets

[{'contributors': None,
  'coordinates': None,
  'created_at': 'Fri Dec 08 03:19:37 +0000 2017',
  'entities': {'hashtags': [], 'symbols': [], 'urls': [], 'user_mentions': []},
  'favorite_count': 0,
  'favorited': False,
  'geo': None,
  'id': 938971297169551360,
  'id_str': '938971297169551360',
  'in_reply_to_screen_name': None,
  'in_reply_to_status_id': None,
  'in_reply_to_status_id_str': None,
  'in_reply_to_user_id': None,
  'in_reply_to_user_id_str': None,
  'is_quote_status': False,
  'lang': 'en',
  'place': None,
  'retweet_count': 0,
  'retweeted': False,
  'source': '<a href="https://github.com/thugh001" rel="nofollow">UCI_Bootcamp_TLH84</a>',
  'text': 'Count your age by friends, not years. Count your life by smiles, not tears. - John Lennon',
  'truncated': False,
  'user': {'contributors_enabled': False,
   'created_at': 'Sat Dec 02 18:01:07 +0000 2017',
   'default_profile': True,
   'default_profile_image': True,
   'description': '',
   'entities': {'description': {

In [170]:
#Create empty list to hold all tweet data 
tweet_list = []

In [171]:
#Loop through all news outlets 
for news in news_outlets:
    counter = 1
    
    #Get 100 tweets from each news outlet home feed
    public_tweets = api.user_timeline(news, count=100)
    
    #Loop tweets and perform sentiment analysis on each tweet
    for tweet in public_tweets:
    
        #Run Vader Analysis on each tweet
        compound = analyzer.polarity_scores(tweet["text"])["compound"]
        pos = analyzer.polarity_scores(tweet["text"])["pos"]
        neu = analyzer.polarity_scores(tweet["text"])["neu"]
        neg = analyzer.polarity_scores(tweet["text"])["neg"]
        
        #Add tweet data to the appropriate array
        tweet_dict = {'Source': news,'Date': tweet["created_at"],'Text': tweet["text"],'Compound Score': compound,'Positive Score': pos, 'Neutral Score': neu,'Negative Score': neg, 'Tweets Ago': counter}      
        
        #Append tweet dictionary to empty list that will later be turned into a dataframe
        tweet_list.append(tweet_dict)
        
        #Increment counter to assess number of tweets ago
        counter +=1 

In [172]:
#Review tweet list that holds tweet dictionary
tweet_list

[{'Compound Score': 0.0,
  'Date': 'Wed Dec 13 20:00:10 +0000 2017',
  'Negative Score': 0.0,
  'Neutral Score': 1.0,
  'Positive Score': 0.0,
  'Source': '@BBC',
  'Text': "London's Euston station will be turned into a shelter for the homeless on Christmas Day. ❤️️🎁… https://t.co/Y834ULzLu5",
  'Tweets Ago': 1},
 {'Compound Score': 0.3384,
  'Date': 'Wed Dec 13 19:31:04 +0000 2017',
  'Negative Score': 0.0,
  'Neutral Score': 0.854,
  'Positive Score': 0.146,
  'Source': '@BBC',
  'Text': '🐘 Why you must never pee in front of elephants... and more surprising facts. 👉 https://t.co/Bz8ncFjAfa https://t.co/o2vs4NJDK0',
  'Tweets Ago': 2},
 {'Compound Score': 0.2263,
  'Date': 'Wed Dec 13 19:00:02 +0000 2017',
  'Negative Score': 0.0,
  'Neutral Score': 0.917,
  'Positive Score': 0.083,
  'Source': '@BBC',
  'Text': "It’s spicy, it's tangy &amp; it'll warm you right up. 🍲😋\nGet the full recipe for @Rick_Stein's Sopa Azteca here. 👉… https://t.co/jLcGMawMgO",
  'Tweets Ago': 3},
 {'Compound

In [173]:
#Create a data frame that stores tweet source account name, text, date, and sentiment scores 
df = pd.DataFrame.from_dict(tweet_list)
df.head()

,Compound Score,Date,Negative Score,Neutral Score,Positive Score,Source,Text,Tweets Ago
0,0.0000,Wed Dec 13 20:00:10 +0000 2017,0.000,1.000,0.000,@BBC,London's Euston station will be turned into a ...,1
1,0.3384,Wed Dec 13 19:31:04 +0000 2017,0.000,0.854,0.146,@BBC,🐘 Why you must never pee in front of elephants...,2
2,0.2263,Wed Dec 13 19:00:02 +0000 2017,0.000,0.917,0.083,@BBC,"It’s spicy, it's tangy &amp; it'll warm you ri...",3
3,-0.4215,Wed Dec 13 18:32:04 +0000 2017,0.189,0.811,0.000,@BBC,Princess Margaret broke all the royal rules lo...,4
4,0.1280,Wed Dec 13 18:00:12 +0000 2017,0.000,0.919,0.081,@BBC,"Còsagach: A feeling of being snug, sheltered o...",5


In [174]:
#Re-order data frame columns 
sentiment_df = df[['Source', 'Date', 'Text', 'Compound Score', 'Positive Score', 'Neutral Score', 'Negative Score', 'Tweets Ago']]

#Print dataframe head
sentiment_df.head()

,Source,Date,Text,Compound Score,Positive Score,Neutral Score,Negative Score,Tweets Ago
0,@BBC,Wed Dec 13 20:00:10 +0000 2017,London's Euston station will be turned into a ...,0.0000,0.000,1.000,0.000,1
1,@BBC,Wed Dec 13 19:31:04 +0000 2017,🐘 Why you must never pee in front of elephants...,0.3384,0.146,0.854,0.000,2
2,@BBC,Wed Dec 13 19:00:02 +0000 2017,"It’s spicy, it's tangy &amp; it'll warm you ri...",0.2263,0.083,0.917,0.000,3
3,@BBC,Wed Dec 13 18:32:04 +0000 2017,Princess Margaret broke all the royal rules lo...,-0.4215,0.000,0.811,0.189,4
4,@BBC,Wed Dec 13 18:00:12 +0000 2017,"Còsagach: A feeling of being snug, sheltered o...",0.1280,0.081,0.919,0.000,5


In [175]:
#Verify there are 500 tweets
len(sentiment_df)

500

In [176]:
#Save CSV file of all weather data retrieved
sentiment_df.to_csv("Sentiment.csv")

In [1]:
# Build a scatter plot of sentiments for each news outlet
plt.scatter(sentiment_df["Latitude"], sentiment_df["Temperature"], marker="o", facecolors="skyblue", label=" ", edgecolors="black", s="Compound Score", alpha=1)

# X and Y Limits
plt.ylim(1.0, -1.0)
plt.xlim(105, -5.0)

# Incorporate the other graph properties
plt.title("Sentiment Analysis of Media Tweets (12/14/17)")
plt.ylabel("Tweet Polarity")
plt.xlabel("Tweets Ago")
plt.grid(True)

# Save the figure
plt.savefig("Sentiment_Analysis.png")

# Show plot
plt.show()

NameError: name 'plt' is not defined

In [ ]:
a